In [1]:
import os.path as op
import mne
from scripts.processing.library.config import study_path, subjects_dir, N_JOBS, spacing
%matplotlib inline    

In [ ]:
def run_bem(subject_id):
    subject = "sub%03d" % subject_id
    print(subject)
    subjects_dir = op.join(study_path, 'subjects')
    print(subjects_dir)
    
    #BEM ESTIMATION
    conductivity = (0.3, 0.006, 0.3)  # for three layer (EEG)

    # BEM MODEL FILE
    fname_bem_surfaces = op.join(subjects_dir, subject, 'bem',
                                     '%s-5120-bem-3layers.fif' % (subject))
    # BEM SOLUTION FILE
    fname_bem = op.join(subjects_dir, subject, 'bem', 
                    '%s-5120-bem-sol-3layers.fif' % (subject))
    # CHECK UBICATION OF THE FILES

    print(fname_bem_surfaces)
    print(fname_bem)
    

    
    try:
        if not op.isfile(fname_bem_surfaces):
            #WRITE BEM SURFACES
            print("\nWRITING BEM SURFACES...")        
            model = mne.make_bem_model(subject=subject, ico=4,
                                       conductivity=conductivity,
                                       subjects_dir=subjects_dir)
           
            mne.write_bem_surfaces(fname_bem_surfaces, model)

        if not op.isfile(fname_bem):
            # WRITE BEM SOLUTION
            bem_model = mne.read_bem_surfaces(fname_bem_surfaces)
            print("\n***************MAKING BEM SOLUTION...")
            bem = mne.make_bem_solution(bem_model)
            print("\n***************WRITING BEM SOLUTION...")
            mne.write_bem_solution(fname_bem, bem)
        
        # Create the surface source space
        print("\n***************CREATING SOURCE SPACE...")
        fname_src = op.join(subjects_dir, subject, 'bem', '%s-%s-EEG-src.fif'
                        % (subject, spacing))
        if not op.isfile(fname_src):
            print('  Setting up source space')
            src = mne.setup_source_space(subject, spacing,
                                     subjects_dir=subjects_dir)
            mne.write_source_spaces(fname_src, src)
        print("\n---------------READY--------------------")
    except:
        return 1


In [ ]:
subject_id = 2
bad_subjects_3_layers = []
good_subjects_3_layers = []

while subject_id < 19:
    salida = run_bem(subject_id)
    print("\n************ OUT: ", salida)
    if salida == 1:
        print("\n---------------------PROBLEM WITH SUBJECT: ",subject_id)
        bad_subjects_3_layers.append(subject_id)
        subject_id+=1
    else:
        print("\n---------------------NO PROBLEM WITH SUBJECT: ", subject_id)
        good_subjects_3_layers.append(subject_id)
        subject_id+=1

# TENER EN CUENTA ESTAS SALIDAS PARA FORWARD
print("\n\nBAD SUBJECTS FOR 3 LAYERS (EEG): ", bad_subjects_3_layers)
print("\n\nGOOD SUBJECTS FOR 3 LAYERS (EEG): ", good_subjects_3_layers)

## CASO SUBJECT 7

In [ ]:
run_bem(subject_id=7)

## <font color="red">ERRORES A CONSIDERAR (verlos en check_surfaces!)</font>

ERROR: Checking that outer skull surface is inside outer skin  surface...

RuntimeError: Surface outer skull is not completely inside surface outer skin 

## CASO SUBJECT 9

In [ ]:
run_bem(subject_id=9)

In [ ]:
run_bem(subject_id=10)